In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/length-of-stay-prediction/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2015.csv


In [2]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv("/kaggle/input/length-of-stay-prediction/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2015.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df["Facility Name"].value_counts()

Mount Sinai Hospital                                              55874
North Shore University Hospital                                   47664
New York Presbyterian Hospital - Columbia Presbyterian Center     47495
New York Presbyterian Hospital - New York Weill Cornell Center    43148
Montefiore Medical Center - Henry & Lucy Moses Div                42694
                                                                  ...  
Cuba Memorial Hospital Inc                                           61
Catskill Regional Medical Center - G. Hermann Site                   50
Monroe Community Hospital                                            24
Westfield Memorial Hospital Inc                                      11
Albany Medical Center - South Clinical Campus                         7
Name: Facility Name, Length: 224, dtype: int64

In [5]:
df.head()

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,5333.90,4818.42
1,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,NaN,NaN,0,N,Y,4865.99,4588.78
2,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,Private Health Insurance,NaN,0,N,Y,5901.54,5559.56
3,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,F,White,Not Span/Hispanic,...,Minor,Medical,Medicare,NaN,NaN,0,N,Y,3619.08,3567.25
4,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,M,White,Not Span/Hispanic,...,Moderate,Medical,Medicare,Blue Cross/Blue Shield,Private Health Insurance,0,N,Y,3185.87,3167.89


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346931 entries, 0 to 2346930
Data columns (total 34 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Health Service Area                  object 
 1   Hospital County                      object 
 2   Operating Certificate Number         float64
 3   Facility Id                          float64
 4   Facility Name                        object 
 5   Age Group                            object 
 6   Zip Code - 3 digits                  object 
 7   Gender                               object 
 8   Race                                 object 
 9   Ethnicity                            object 
 10  Length of Stay                       object 
 11  Type of Admission                    object 
 12  Patient Disposition                  object 
 13  Discharge Year                       int64  
 14  CCS Diagnosis Code                   int64  
 15  CCS Diagnosis Description       

In [7]:
List =["Health Service Area","Hospital County","Age Group","Zip Code - 3 digits","Gender","Race","Ethnicity","Length of Stay","Type of Admission",                    
"Patient Disposition","CCS Diagnosis Description","CCS Procedure Description","APR DRG Description","APR MDC Description","APR Severity of Illness Description","APR Risk of Mortality","APR Medical Surgical Description","Payment Typology 1"                   
,"Payment Typology 2","Payment Typology 3","Abortion Edit Indicator","Emergency Department Indicator"]
for col in List :
    df[col] = df[col].astype('category')

#"Health Service Area"

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346931 entries, 0 to 2346930
Data columns (total 34 columns):
 #   Column                               Dtype   
---  ------                               -----   
 0   Health Service Area                  category
 1   Hospital County                      category
 2   Operating Certificate Number         float64 
 3   Facility Id                          float64 
 4   Facility Name                        object  
 5   Age Group                            category
 6   Zip Code - 3 digits                  category
 7   Gender                               category
 8   Race                                 category
 9   Ethnicity                            category
 10  Length of Stay                       category
 11  Type of Admission                    category
 12  Patient Disposition                  category
 13  Discharge Year                       int64   
 14  CCS Diagnosis Code                   int64   
 15  CCS Diagnosis D

In [9]:
col=["Health Service Area"
 ,
 "CCS Diagnosis Description",
 "CCS Procedure Description",
"APR DRG Description",
"APR MDC Description",
"APR Severity of Illness Description"]
df = df.drop(col, axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346931 entries, 0 to 2346930
Data columns (total 28 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   Hospital County                   category
 1   Operating Certificate Number      float64 
 2   Facility Id                       float64 
 3   Facility Name                     object  
 4   Age Group                         category
 5   Zip Code - 3 digits               category
 6   Gender                            category
 7   Race                              category
 8   Ethnicity                         category
 9   Length of Stay                    category
 10  Type of Admission                 category
 11  Patient Disposition               category
 12  Discharge Year                    int64   
 13  CCS Diagnosis Code                int64   
 14  CCS Procedure Code                int64   
 15  APR DRG Code                      int64   
 16  APR MDC Code      

##clint 1: Albany

In [11]:
Albany_df=df[df["Facility Name"]== 'Albany Medical Center Hospital']


In [12]:
Albany_df

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
490252,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,120,F,White,Not Span/Hispanic,1,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,10397.13,3610.96
491806,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,120,F,White,Not Span/Hispanic,1,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,9976.77,2263.12
492215,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,122,F,Black/African American,Not Span/Hispanic,1,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,6042.14,1968.23
492519,Albany,101000.0,1.0,Albany Medical Center Hospital,50 to 69,120,F,White,Not Span/Hispanic,1,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,17876.70,4662.19
493215,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,124,M,Other Race,Unknown,1,...,Minor,Medical,Private Health Insurance,NaN,NaN,0,N,N,9919.29,3418.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536364,Albany,101000.0,1.0,Albany Medical Center Hospital,18 to 29,121,M,White,Not Span/Hispanic,1,...,Minor,Surgical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,16726.52,5145.79
536365,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,121,F,Other Race,Unknown,2,...,Minor,Medical,Medicaid,NaN,NaN,4700,N,N,3829.74,947.76
536366,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,124,M,Other Race,Unknown,3,...,Minor,Medical,Medicaid,NaN,NaN,3200,N,N,7209.25,1661.29
536367,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,120,F,Other Race,Unknown,83,...,Moderate,Medical,Medicaid,NaN,NaN,900,N,N,665166.90,165272.09


In [13]:
Albany_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36281 entries, 490252 to 536368
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   36281 non-null  category
 1   Operating Certificate Number      36281 non-null  float64 
 2   Facility Id                       36281 non-null  float64 
 3   Facility Name                     36281 non-null  object  
 4   Age Group                         36281 non-null  category
 5   Zip Code - 3 digits               35772 non-null  category
 6   Gender                            36281 non-null  category
 7   Race                              36281 non-null  category
 8   Ethnicity                         36281 non-null  category
 9   Length of Stay                    36281 non-null  category
 10  Type of Admission                 36281 non-null  category
 11  Patient Disposition               36281 non-null

In [14]:
 #   simple mapping on binary features.
Albany_df['Abortion Edit Indicator'] = Albany_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Albany_df['Emergency Department Indicator'] = Albany_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits","Payment Typology 2"        
   ,"Payment Typology 3"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.        
   
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'. 
for col in list :
 Albany_df[col]= label_encoder.fit_transform(Albany_df[col])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
Albany_df=Albany_df.drop("Facility Name", axis=1)

In [17]:
Albany_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36281 entries, 490252 to 536368
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   36281 non-null  int64   
 1   Operating Certificate Number      36281 non-null  float64 
 2   Facility Id                       36281 non-null  float64 
 3   Age Group                         36281 non-null  int64   
 4   Zip Code - 3 digits               36281 non-null  int64   
 5   Gender                            36281 non-null  int64   
 6   Race                              36281 non-null  int64   
 7   Ethnicity                         36281 non-null  int64   
 8   Length of Stay                    36281 non-null  category
 9   Type of Admission                 36281 non-null  int64   
 10  Patient Disposition               36281 non-null  int64   
 11  Discharge Year                    36281 non-null

In [18]:
Albany_df["Length of Stay"]=pd.to_numeric(Albany_df["Length of Stay"],errors='coerce')


In [19]:
Albany_df.dropna(inplace=True)

In [20]:
x = Albany_df.drop(["Length of Stay"], axis =1)
y = Albany_df["Length of Stay"]

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [22]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)


In [23]:
r2 = r2_score(y_test, y_pred)
r2 

0.7478105920657923

In [24]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

2.3205026865513774

In [25]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

16.946608108006014

In [26]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

4.116625815884413

In [27]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [28]:
r2 = r2_score(y_test, y_pred1)
r2 


0.730292669976124

In [29]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

2.380819108363997

In [30]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

18.123776344182055

In [31]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

4.257202877968356

In [32]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [33]:
r2 = r2_score(y_test, y_pred3)
r2 

0.7478105537618742

In [34]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

2.3204943412883794

In [35]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

16.946610681950318

In [36]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

4.116626128512318

##Clent 2 : St Josephs Hospital Health Center

In [37]:
Josephs_df=df[df["Facility Name"]== 'St Josephs Hospital Health Center']

In [38]:
#   simple mapping on binary features.
Josephs_df['Abortion Edit Indicator'] = Josephs_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Josephs_df['Emergency Department Indicator'] = Josephs_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [39]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits","Payment Typology 2"        
   ,"Payment Typology 3"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.        
   
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'. 
for col in list :
    Josephs_df[col]= label_encoder.fit_transform(Josephs_df[col])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [40]:
Josephs_df=Josephs_df.drop("Facility Name", axis=1)


In [41]:
Josephs_df["Length of Stay"]=pd.to_numeric(Josephs_df["Length of Stay"],errors='coerce')


In [42]:
Josephs_df.dropna(inplace=True)

In [43]:
Josephs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25626 entries, 355968 to 433177
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   25626 non-null  int64  
 1   Operating Certificate Number      25626 non-null  float64
 2   Facility Id                       25626 non-null  float64
 3   Age Group                         25626 non-null  int64  
 4   Zip Code - 3 digits               25626 non-null  int64  
 5   Gender                            25626 non-null  int64  
 6   Race                              25626 non-null  int64  
 7   Ethnicity                         25626 non-null  int64  
 8   Length of Stay                    25626 non-null  float64
 9   Type of Admission                 25626 non-null  int64  
 10  Patient Disposition               25626 non-null  int64  
 11  Discharge Year                    25626 non-null  int64  
 12

In [44]:
x = Josephs_df.drop(["Length of Stay"], axis =1)
y = Josephs_df["Length of Stay"]

In [45]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [46]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [47]:
r2 = r2_score(y_test, y_pred)
r2

0.6574754436024834

In [48]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

2.0719697846829397

In [49]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

13.044654985643254

In [50]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.6117384990670702

In [51]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [52]:
r2 = r2_score(y_test, y_pred1)
r2 

0.5810415145070502

In [53]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

2.455629978876319

In [54]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

15.95555353473856

In [55]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.994440328098363

In [56]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [57]:
r2 = r2_score(y_test, y_pred3)
r2 

0.6574730902842871

In [58]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

2.071950047482412

In [59]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

13.0447446090684

In [60]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.6117509062874755

##client 3 : Bronx-Lebanon 

In [61]:
bronx=['Bronx-Lebanon Hospital Center - Fulton Division', 'Bronx-Lebanon Hospital Center - Concourse Division']
Bronx_df1=df[df["Facility Name"]== 'Bronx-Lebanon Hospital Center - Fulton Division']

In [62]:
Bronx_df2=df[df["Facility Name"]== 'Bronx-Lebanon Hospital Center - Concourse Division']

In [63]:
l = [Bronx_df1,Bronx_df2]

In [64]:
Bronx_df= pd.concat(l)
display(Bronx_df)

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
905707,Bronx,7000001.0,1164.0,Bronx-Lebanon Hospital Center - Fulton Division,50 to 69,104,M,White,Not Span/Hispanic,3,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,9004.46,7831.22
905777,Bronx,7000001.0,1164.0,Bronx-Lebanon Hospital Center - Fulton Division,18 to 29,104,M,Other Race,Unknown,11,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,18290.67,16470.61
911991,Bronx,7000001.0,1164.0,Bronx-Lebanon Hospital Center - Fulton Division,50 to 69,104,F,Other Race,Unknown,6,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,10959.98,10114.66
912090,Bronx,7000001.0,1164.0,Bronx-Lebanon Hospital Center - Fulton Division,18 to 29,104,M,Other Race,Unknown,22,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,32614.70,29786.31
912212,Bronx,7000001.0,1164.0,Bronx-Lebanon Hospital Center - Fulton Division,50 to 69,104,M,Other Race,Spanish/Hispanic,3,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,7838.81,7002.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065404,Bronx,7000001.0,1178.0,Bronx-Lebanon Hospital Center - Concourse Divi...,18 to 29,104,F,Other Race,Unknown,2,...,Minor,Medical,Medicaid,Medicaid,NaN,0,N,N,7194.54,8400.69
1065405,Bronx,7000001.0,1178.0,Bronx-Lebanon Hospital Center - Concourse Divi...,30 to 49,104,F,Other Race,Unknown,2,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,9129.09,9080.43
1065406,Bronx,7000001.0,1178.0,Bronx-Lebanon Hospital Center - Concourse Divi...,0 to 17,104,F,Other Race,Spanish/Hispanic,3,...,Minor,Medical,Private Health Insurance,Private Health Insurance,Medicaid,0,N,Y,14112.64,10711.62
1065407,Bronx,7000001.0,1178.0,Bronx-Lebanon Hospital Center - Concourse Divi...,70 or Older,104,F,Black/African American,Not Span/Hispanic,5,...,Major,Medical,Medicare,Medicare,Medicaid,0,N,Y,14171.30,16734.73


In [65]:
Bronx_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31978 entries, 905707 to 1065408
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   31978 non-null  category
 1   Operating Certificate Number      31978 non-null  float64 
 2   Facility Id                       31978 non-null  float64 
 3   Facility Name                     31978 non-null  object  
 4   Age Group                         31978 non-null  category
 5   Zip Code - 3 digits               31061 non-null  category
 6   Gender                            31978 non-null  category
 7   Race                              31978 non-null  category
 8   Ethnicity                         31978 non-null  category
 9   Length of Stay                    31978 non-null  category
 10  Type of Admission                 31978 non-null  category
 11  Patient Disposition               31978 non-nul

In [66]:
Bronx_df = Bronx_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)

In [67]:
for col in Bronx_df.columns:
    print( col,':', Bronx_df[col].nunique() )#Num of unique values
    print(Bronx_df[col].value_counts().nlargest(5)) #Most 5 frequent values
    print('\n' + '*' * 20 + '\n')

Hospital County : 1
Bronx          31978
Albany             0
Schenectady        0
Onondaga           0
Ontario            0
Name: Hospital County, dtype: int64

********************

Operating Certificate Number : 1
7000001.0    31978
Name: Operating Certificate Number, dtype: int64

********************

Facility Id : 2
1178.0    27369
1164.0     4609
Name: Facility Id, dtype: int64

********************

Facility Name : 2
Bronx-Lebanon Hospital Center - Concourse Division    27369
Bronx-Lebanon Hospital Center - Fulton Division        4609
Name: Facility Name, dtype: int64

********************

Age Group : 5
50 to 69       10654
30 to 49        8444
0 to 17         4601
70 or Older     4206
18 to 29        4073
Name: Age Group, dtype: int64

********************

Zip Code - 3 digits : 20
104    28864
100     1010
OOS      480
112      358
114      101
Name: Zip Code - 3 digits, dtype: int64

********************

Gender : 2
F    17142
M    14836
U        0
Name: Gender, dtype: int6

In [68]:
 #   simple mapping on binary features.
Bronx_df['Abortion Edit Indicator'] = Bronx_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Bronx_df['Emergency Department Indicator'] = Bronx_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [69]:
Bronx_df["Length of Stay"]=pd.to_numeric(Bronx_df["Length of Stay"],errors='coerce')


In [70]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits",'Facility Name']  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
  Bronx_df[col]= label_encoder.fit_transform(Bronx_df[col])

In [71]:
Bronx_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 31978 entries, 905707 to 1065408
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   31978 non-null  int64  
 1   Operating Certificate Number      31978 non-null  float64
 2   Facility Id                       31978 non-null  float64
 3   Facility Name                     31978 non-null  int64  
 4   Age Group                         31978 non-null  int64  
 5   Zip Code - 3 digits               31978 non-null  int64  
 6   Gender                            31978 non-null  int64  
 7   Race                              31978 non-null  int64  
 8   Ethnicity                         31978 non-null  int64  
 9   Length of Stay                    31951 non-null  float64
 10  Type of Admission                 31978 non-null  int64  
 11  Patient Disposition               31978 non-null  int64  
 1

In [72]:
 Bronx_df.dropna(inplace=True)


In [73]:
x = Bronx_df.drop(["Length of Stay"], axis =1)
y = Bronx_df["Length of Stay"]

In [74]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [75]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [76]:
r2 = r2_score(y_test, y_pred)
r2 

0.8376128349444951

In [77]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

1.5959481188252358

In [78]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

9.830664824520944

In [79]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.1353891025709943

In [80]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [81]:
r2 = r2_score(y_test, y_pred1)
r2 

0.8157805173305335

In [82]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.739528477025208

In [83]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

11.152359163676266

In [84]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.3395148096207428

In [85]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [86]:
r2 = r2_score(y_test, y_pred3)
r2 

0.8376129585591974

In [87]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.5959277579640252

In [88]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

9.830657341080334

In [89]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.1353879091876866

##Client 4 : United Health

In [90]:
# 4  'United Health Services Hospitals Inc. - Binghamton General Hospital',
# 4  'United Health Services Hospitals Inc. - Wilson Medical Center',
UH_df1=df[df["Facility Name"]== 'United Health Services Hospitals Inc. - Wilson Medical Center']
UH_df2=df[df["Facility Name"]== 'United Health Services Hospitals Inc. - Binghamton General Hospital']

l=[UH_df1,UH_df2]
UH_df= pd.concat(l)
display(UH_df)

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
327335,Broome,303001.0,58.0,United Health Services Hospitals Inc. - Wilson...,30 to 49,139,M,Other Race,Spanish/Hispanic,120 +,...,Extreme,Surgical,Medicaid,NaN,NaN,0,N,Y,1032046.99,429514.66
327336,Broome,303001.0,58.0,United Health Services Hospitals Inc. - Wilson...,50 to 69,138,F,White,Not Span/Hispanic,120 +,...,Extreme,Surgical,Private Health Insurance,NaN,NaN,0,N,Y,1091681.06,378826.75
327337,Broome,303001.0,58.0,United Health Services Hospitals Inc. - Wilson...,30 to 49,137,M,White,Not Span/Hispanic,96,...,Extreme,Surgical,Medicaid,Medicaid,NaN,0,N,Y,460332.46,161178.76
327338,Broome,303001.0,58.0,United Health Services Hospitals Inc. - Wilson...,70 or Older,137,M,Other Race,Not Span/Hispanic,1,...,Moderate,Surgical,Medicare,Medicare,Blue Cross/Blue Shield,0,N,N,17877.80,6651.53
327339,Broome,303001.0,58.0,United Health Services Hospitals Inc. - Wilson...,50 to 69,137,F,Black/African American,Not Span/Hispanic,74,...,Extreme,Surgical,Medicaid,NaN,NaN,0,N,Y,399707.26,138410.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316422,Broome,303001.0,42.0,United Health Services Hospitals Inc. - Bingha...,30 to 49,137,F,White,Not Span/Hispanic,2,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,5885.90,2617.98
316423,Broome,303001.0,42.0,United Health Services Hospitals Inc. - Bingha...,30 to 49,137,F,White,Not Span/Hispanic,1,...,Minor,Medical,Medicaid,Medicaid,NaN,0,N,Y,3320.30,1401.22
316424,Broome,303001.0,42.0,United Health Services Hospitals Inc. - Bingha...,50 to 69,137,F,White,Not Span/Hispanic,3,...,Minor,Medical,Medicare,Medicare,Federal/State/Local/VA,0,N,N,4251.50,2673.10
316425,Broome,303001.0,42.0,United Health Services Hospitals Inc. - Bingha...,18 to 29,138,F,White,Not Span/Hispanic,2,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,5082.30,2275.24


In [91]:
UH_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17661 entries, 327335 to 316426
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   17661 non-null  category
 1   Operating Certificate Number      17661 non-null  float64 
 2   Facility Id                       17661 non-null  float64 
 3   Facility Name                     17661 non-null  object  
 4   Age Group                         17661 non-null  category
 5   Zip Code - 3 digits               17460 non-null  category
 6   Gender                            17661 non-null  category
 7   Race                              17661 non-null  category
 8   Ethnicity                         17661 non-null  category
 9   Length of Stay                    17661 non-null  category
 10  Type of Admission                 17661 non-null  category
 11  Patient Disposition               17661 non-null

In [92]:
UH_df = UH_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)


In [93]:
for col in UH_df.columns:
    print( col,':', UH_df[col].nunique() )#Num of unique values
    print(UH_df[col].value_counts().nlargest(5)) #Most 5 frequent values
    print('\n' + '*' * 20 + '\n')

Hospital County : 1
Broome         17661
Albany             0
Schenectady        0
Onondaga           0
Ontario            0
Name: Hospital County, dtype: int64

********************

Operating Certificate Number : 1
303001.0    17661
Name: Operating Certificate Number, dtype: int64

********************

Facility Id : 2
58.0    13318
42.0     4343
Name: Facility Id, dtype: int64

********************

Facility Name : 2
United Health Services Hospitals Inc. - Wilson Medical Center          13318
United Health Services Hospitals Inc. - Binghamton General Hospital     4343
Name: Facility Name, dtype: int64

********************

Age Group : 5
70 or Older    5807
50 to 69       5171
30 to 49       2961
0 to 17        1870
18 to 29       1852
Name: Age Group, dtype: int64

********************

Zip Code - 3 digits : 28
137    7150
139    4731
138    4070
OOS    1014
148     125
Name: Zip Code - 3 digits, dtype: int64

********************

Gender : 2
F    9398
M    8263
U       0
Name: Gen

In [94]:
 #   simple mapping on binary features.
UH_df['Abortion Edit Indicator'] = UH_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
UH_df['Emergency Department Indicator'] = UH_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [95]:
UH_df["Length of Stay"]=pd.to_numeric(UH_df["Length of Stay"],errors='coerce')


In [96]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits",'Facility Name']  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
  UH_df[col]= label_encoder.fit_transform(UH_df[col])


In [97]:
UH_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17661 entries, 327335 to 316426
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   17661 non-null  int64  
 1   Operating Certificate Number      17661 non-null  float64
 2   Facility Id                       17661 non-null  float64
 3   Facility Name                     17661 non-null  int64  
 4   Age Group                         17661 non-null  int64  
 5   Zip Code - 3 digits               17661 non-null  int64  
 6   Gender                            17661 non-null  int64  
 7   Race                              17661 non-null  int64  
 8   Ethnicity                         17661 non-null  int64  
 9   Length of Stay                    17645 non-null  float64
 10  Type of Admission                 17661 non-null  int64  
 11  Patient Disposition               17661 non-null  int64  
 12

In [98]:
 UH_df.dropna(inplace=True)


In [99]:
x = UH_df.drop(["Length of Stay"], axis =1)
y = UH_df["Length of Stay"]

In [100]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


In [101]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [102]:
r2 = r2_score(y_test, y_pred)
r2 

0.7525860452304862

In [103]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

2.0684639957341076

In [104]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

12.376363549957983

In [105]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.5180056210810666

In [106]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [107]:
r2 = r2_score(y_test, y_pred1)
r2 

0.7149297869535722

In [108]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

2.2303833727619162

In [109]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

14.260038796976138

In [110]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.7762466546792384

In [111]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [112]:
r2 = r2_score(y_test, y_pred3)
r2 

0.7525870132619732

In [113]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

2.0684430974297547

In [114]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

12.376315126215594

In [115]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.517998738802445

##Client 6 : Glens Falls Hospital

In [116]:
Glens_df=df[df["Facility Name"]== 'Glens Falls Hospital']

In [117]:
Glens_df

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
628756,Warren,5601000.0,1005.0,Glens Falls Hospital,30 to 49,128,F,White,Not Span/Hispanic,5,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,16901.84,5444.54
628825,Warren,5601000.0,1005.0,Glens Falls Hospital,50 to 69,120,F,White,Not Span/Hispanic,4,...,Moderate,Medical,Medicare,NaN,NaN,0,N,N,15354.59,5499.39
629035,Warren,5601000.0,1005.0,Glens Falls Hospital,18 to 29,128,M,White,Not Span/Hispanic,2,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,8413.10,2565.18
629070,Warren,5601000.0,1005.0,Glens Falls Hospital,50 to 69,128,F,White,Not Span/Hispanic,1,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,8102.00,2008.34
629175,Warren,5601000.0,1005.0,Glens Falls Hospital,70 or Older,128,F,White,Not Span/Hispanic,6,...,Major,Medical,Medicare,NaN,NaN,0,N,N,21480.74,7279.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664926,Warren,5601000.0,1005.0,Glens Falls Hospital,30 to 49,128,F,White,Not Span/Hispanic,1,...,Minor,Medical,Medicare,NaN,NaN,0,N,Y,14033.73,4471.53
664927,Warren,5601000.0,1005.0,Glens Falls Hospital,70 or Older,128,F,White,Not Span/Hispanic,1,...,Minor,Surgical,Medicare,NaN,NaN,0,N,Y,44134.98,12744.49
664928,Warren,5601000.0,1005.0,Glens Falls Hospital,30 to 49,128,M,White,Not Span/Hispanic,1,...,Minor,Medical,Private Health Insurance,NaN,NaN,0,N,N,13780.26,3192.12
664929,Warren,5601000.0,1005.0,Glens Falls Hospital,50 to 69,128,M,White,Not Span/Hispanic,1,...,Moderate,Medical,Private Health Insurance,NaN,NaN,0,N,N,7560.97,2705.89


In [118]:
Glens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14072 entries, 628756 to 664930
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   14072 non-null  category
 1   Operating Certificate Number      14072 non-null  float64 
 2   Facility Id                       14072 non-null  float64 
 3   Facility Name                     14072 non-null  object  
 4   Age Group                         14072 non-null  category
 5   Zip Code - 3 digits               13957 non-null  category
 6   Gender                            14072 non-null  category
 7   Race                              14072 non-null  category
 8   Ethnicity                         14072 non-null  category
 9   Length of Stay                    14072 non-null  category
 10  Type of Admission                 14072 non-null  category
 11  Patient Disposition               14072 non-null

In [119]:
Glens_df.head()

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
628756,Warren,5601000.0,1005.0,Glens Falls Hospital,30 to 49,128,F,White,Not Span/Hispanic,5,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,16901.84,5444.54
628825,Warren,5601000.0,1005.0,Glens Falls Hospital,50 to 69,120,F,White,Not Span/Hispanic,4,...,Moderate,Medical,Medicare,NaN,NaN,0,N,N,15354.59,5499.39
629035,Warren,5601000.0,1005.0,Glens Falls Hospital,18 to 29,128,M,White,Not Span/Hispanic,2,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,8413.10,2565.18
629070,Warren,5601000.0,1005.0,Glens Falls Hospital,50 to 69,128,F,White,Not Span/Hispanic,1,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,8102.00,2008.34
629175,Warren,5601000.0,1005.0,Glens Falls Hospital,70 or Older,128,F,White,Not Span/Hispanic,6,...,Major,Medical,Medicare,NaN,NaN,0,N,N,21480.74,7279.17


In [120]:
Glens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14072 entries, 628756 to 664930
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   14072 non-null  category
 1   Operating Certificate Number      14072 non-null  float64 
 2   Facility Id                       14072 non-null  float64 
 3   Facility Name                     14072 non-null  object  
 4   Age Group                         14072 non-null  category
 5   Zip Code - 3 digits               13957 non-null  category
 6   Gender                            14072 non-null  category
 7   Race                              14072 non-null  category
 8   Ethnicity                         14072 non-null  category
 9   Length of Stay                    14072 non-null  category
 10  Type of Admission                 14072 non-null  category
 11  Patient Disposition               14072 non-null

In [121]:
Glens_df = Glens_df.drop("Facility Name", axis=1)

In [122]:
Glens_df[["Length of Stay"]]

,Length of Stay
628756,5
628825,4
629035,2
629070,1
629175,6
...,...
664926,1
664927,1
664928,1
664929,1


In [123]:
Glens_df = Glens_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)


In [124]:
for col in Glens_df.columns:
    print( col,':', Glens_df[col].nunique() )#Num of unique values
    print(Glens_df[col].value_counts().nlargest(5)) #Most 5 frequent values
    print('\n' + '*' * 20 + '\n')

Hospital County : 1
Warren      14072
Albany          0
Saratoga        0
Onondaga        0
Ontario         0
Name: Hospital County, dtype: int64

********************

Operating Certificate Number : 1
5601000.0    14072
Name: Operating Certificate Number, dtype: int64

********************

Facility Id : 1
1005.0    14072
Name: Facility Id, dtype: int64

********************

Age Group : 5
70 or Older    4708
50 to 69       3997
30 to 49       2186
0 to 17        1671
18 to 29       1510
Name: Age Group, dtype: int64

********************

Zip Code - 3 digits : 20
128    13284
OOS      197
120      196
129      110
121       80
Name: Zip Code - 3 digits, dtype: int64

********************

Gender : 2
F    7797
M    6275
U       0
Name: Gender, dtype: int64

********************

Race : 3
White                     13610
Other Race                  343
Black/African American      119
Multi-racial                  0
Name: Race, dtype: int64

********************

Ethnicity : 3
Not Span/H

preprocsesing 

In [125]:
 #Glens_df["Length of Stay"] = Glens_df["Length of Stay"].astype('int64')
 #   simple mapping on binary features.
Glens_df['Abortion Edit Indicator'] = Glens_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Glens_df['Emergency Department Indicator'] = Glens_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [126]:
Glens_df["Length of Stay"]=pd.to_numeric(Glens_df["Length of Stay"],errors='coerce')

In [127]:
Glens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14072 entries, 628756 to 664930
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   14072 non-null  category
 1   Operating Certificate Number      14072 non-null  float64 
 2   Facility Id                       14072 non-null  float64 
 3   Age Group                         14072 non-null  category
 4   Zip Code - 3 digits               13957 non-null  category
 5   Gender                            14072 non-null  category
 6   Race                              14072 non-null  category
 7   Ethnicity                         14072 non-null  category
 8   Length of Stay                    14069 non-null  float64 
 9   Type of Admission                 14072 non-null  category
 10  Patient Disposition               14072 non-null  category
 11  Discharge Year                    14072 non-null

In [128]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
  Glens_df[col]= label_encoder.fit_transform(Glens_df[col])

In [129]:
Glens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14072 entries, 628756 to 664930
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   14072 non-null  int64  
 1   Operating Certificate Number      14072 non-null  float64
 2   Facility Id                       14072 non-null  float64
 3   Age Group                         14072 non-null  int64  
 4   Zip Code - 3 digits               14072 non-null  int64  
 5   Gender                            14072 non-null  int64  
 6   Race                              14072 non-null  int64  
 7   Ethnicity                         14072 non-null  int64  
 8   Length of Stay                    14069 non-null  float64
 9   Type of Admission                 14072 non-null  int64  
 10  Patient Disposition               14072 non-null  int64  
 11  Discharge Year                    14072 non-null  int64  
 12

In [130]:
Glens_df.isnull().sum()

Hospital County                     0
Operating Certificate Number        0
Facility Id                         0
Age Group                           0
Zip Code - 3 digits                 0
Gender                              0
Race                                0
Ethnicity                           0
Length of Stay                      3
Type of Admission                   0
Patient Disposition                 0
Discharge Year                      0
CCS Diagnosis Code                  0
CCS Procedure Code                  0
APR DRG Code                        0
APR MDC Code                        0
APR Severity of Illness Code        0
APR Risk of Mortality               0
APR Medical Surgical Description    0
Payment Typology 1                  0
Birth Weight                        0
Abortion Edit Indicator             0
Emergency Department Indicator      0
Total Charges                       0
Total Costs                         0
dtype: int64

In [131]:
Glens_df.dropna(inplace=True)

In [132]:
x = Glens_df.drop(["Length of Stay"], axis =1)
y = Glens_df["Length of Stay"]

In [133]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14069 entries, 628756 to 664930
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   14069 non-null  int64  
 1   Operating Certificate Number      14069 non-null  float64
 2   Facility Id                       14069 non-null  float64
 3   Age Group                         14069 non-null  int64  
 4   Zip Code - 3 digits               14069 non-null  int64  
 5   Gender                            14069 non-null  int64  
 6   Race                              14069 non-null  int64  
 7   Ethnicity                         14069 non-null  int64  
 8   Type of Admission                 14069 non-null  int64  
 9   Patient Disposition               14069 non-null  int64  
 10  Discharge Year                    14069 non-null  int64  
 11  CCS Diagnosis Code                14069 non-null  int64  
 12

In [134]:
y

628756    5.0
628825    4.0
629035    2.0
629070    1.0
629175    6.0
         ... 
664926    1.0
664927    1.0
664928    1.0
664929    1.0
664930    1.0
Name: Length of Stay, Length: 14069, dtype: float64

In [135]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [136]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVR

In [137]:
# Linear Regression
#LR = LinearRegression()
#LR.fit(x_train, y_train)

In [138]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [139]:
r2 = r2_score(y_test, y_pred)
r2 


0.7753107229320534

In [140]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)

In [141]:
mae

1.519038644423741

In [142]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

8.394303923962779

In [143]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.8972925161196237

In [144]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [145]:
r2 = r2_score(y_test, y_pred1)
r2 


0.7507020198871027

In [146]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.5980824331021113

In [147]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

9.313675490018403

In [148]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.0518314976450456

In [149]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [150]:
r2 = r2_score(y_test, y_pred3)
r2 

0.7753072003472253

In [151]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.5188909684172016

In [152]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

8.394435526360677

In [153]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.8973152273028004

##Client 7 : The Burdett Care Center

In [154]:
Burdett_df=df[df["Facility Name"]== 'The Burdett Care Center']

In [155]:
Burdett_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 664931 to 666956
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   2007 non-null   category
 1   Operating Certificate Number      2007 non-null   float64 
 2   Facility Id                       2007 non-null   float64 
 3   Facility Name                     2007 non-null   object  
 4   Age Group                         2007 non-null   category
 5   Zip Code - 3 digits               1990 non-null   category
 6   Gender                            2007 non-null   category
 7   Race                              2007 non-null   category
 8   Ethnicity                         2007 non-null   category
 9   Length of Stay                    2007 non-null   category
 10  Type of Admission                 2007 non-null   category
 11  Patient Disposition               2007 non-null  

In [156]:
for col in Burdett_df.columns:
    print( col,':', Burdett_df[col].nunique() )#Num of unique values
    print(Burdett_df[col].value_counts().nlargest(5)) #Most 5 frequent values
    print('\n' + '*' * 20 + '\n')

Hospital County : 1
Rensselaer    2007
Albany           0
Niagara          0
Onondaga         0
Ontario          0
Name: Hospital County, dtype: int64

********************

Operating Certificate Number : 1
4102004.0    2007
Name: Operating Certificate Number, dtype: int64

********************

Facility Id : 1
9250.0    2007
Name: Facility Id, dtype: int64

********************

Facility Name : 1
The Burdett Care Center    2007
Name: Facility Name, dtype: int64

********************

Age Group : 3
0 to 17        997
18 to 29       623
30 to 49       387
50 to 69         0
70 or Older      0
Name: Age Group, dtype: int64

********************

Zip Code - 3 digits : 7
121    1309
120     466
122     110
123      50
128      45
Name: Zip Code - 3 digits, dtype: int64

********************

Gender : 2
F    1498
M     509
U       0
Name: Gender, dtype: int64

********************

Race : 3
White                     1376
Other Race                 396
Black/African American     235
Multi-ra

In [157]:
 #   simple mapping on binary features.
Burdett_df['Abortion Edit Indicator'] = Burdett_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Burdett_df['Emergency Department Indicator'] = Burdett_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [158]:
Burdett_df.head()

,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
664931,Rensselaer,4102004.0,9250.0,The Burdett Care Center,30 to 49,121,F,White,Not Span/Hispanic,3,...,Minor,Surgical,Private Health Insurance,Self-Pay,Self-Pay,0,0,0,12560.54,7260.24
664932,Rensselaer,4102004.0,9250.0,The Burdett Care Center,18 to 29,120,F,White,Not Span/Hispanic,2,...,Minor,Medical,Medicaid,Self-Pay,Self-Pay,0,0,0,6672.11,3832.18
664933,Rensselaer,4102004.0,9250.0,The Burdett Care Center,18 to 29,120,F,White,Not Span/Hispanic,2,...,Minor,Surgical,Medicaid,Self-Pay,Self-Pay,0,0,0,10598.85,5926.81
664934,Rensselaer,4102004.0,9250.0,The Burdett Care Center,18 to 29,121,F,Black/African American,Not Span/Hispanic,3,...,Minor,Surgical,Blue Cross/Blue Shield,Self-Pay,Self-Pay,0,0,0,17085.67,8402.28
664935,Rensselaer,4102004.0,9250.0,The Burdett Care Center,18 to 29,120,F,White,Not Span/Hispanic,2,...,Minor,Surgical,Medicaid,Self-Pay,Self-Pay,0,0,0,9914.79,5671.25


In [159]:
Burdett_df["Length of Stay"]=pd.to_numeric(Burdett_df["Length of Stay"],errors='coerce')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [160]:
Burdett_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 664931 to 666956
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   2007 non-null   category
 1   Operating Certificate Number      2007 non-null   float64 
 2   Facility Id                       2007 non-null   float64 
 3   Facility Name                     2007 non-null   object  
 4   Age Group                         2007 non-null   category
 5   Zip Code - 3 digits               1990 non-null   category
 6   Gender                            2007 non-null   category
 7   Race                              2007 non-null   category
 8   Ethnicity                         2007 non-null   category
 9   Length of Stay                    2007 non-null   int64   
 10  Type of Admission                 2007 non-null   category
 11  Patient Disposition               2007 non-null  

In [161]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits","Payment Typology 2"        
   ,"Payment Typology 3"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.        
   
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'. 
for col in list :
  Burdett_df[col]= label_encoder.fit_transform(Burdett_df[col])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [162]:
Burdett_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 664931 to 666956
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   2007 non-null   int64  
 1   Operating Certificate Number      2007 non-null   float64
 2   Facility Id                       2007 non-null   float64
 3   Facility Name                     2007 non-null   object 
 4   Age Group                         2007 non-null   int64  
 5   Zip Code - 3 digits               2007 non-null   int64  
 6   Gender                            2007 non-null   int64  
 7   Race                              2007 non-null   int64  
 8   Ethnicity                         2007 non-null   int64  
 9   Length of Stay                    2007 non-null   int64  
 10  Type of Admission                 2007 non-null   int64  
 11  Patient Disposition               2007 non-null   int64  
 12 

In [163]:
Burdett_df=Burdett_df.drop("Facility Name", axis=1)

In [164]:
x = Burdett_df.drop(["Length of Stay"], axis =1)
y = Burdett_df["Length of Stay"]

In [165]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [166]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [167]:
r2 = r2_score(y_test, y_pred)
r2 


0.8723056813594046

In [168]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

0.23125353938413457

In [169]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

0.1265392685345504

In [170]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

0.3557235844508351

In [171]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [172]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

0.3459801547925998

In [173]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

0.3101903514122998

In [174]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

0.5569473506645846

In [175]:
r2 = r2_score(y_test, y_pred1)
r2 


0.6869782318864484

In [176]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [177]:
r2 = r2_score(y_test, y_pred3)
r2 

0.8810130092697138

In [178]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

0.2269137110295729

In [179]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

0.11791070215516318

In [180]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

0.3433812781081158

##Client 8: Putnam Hospital Center

In [181]:
Putnam_df=df[df["Facility Name"]== "Putnam Hospital Center"]

In [182]:
Putnam_df


,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,...,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
700137,Putnam,3950000.0,752.0,Putnam Hospital Center,18 to 29,105,F,Other Race,Unknown,2,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,7555.49,2300.42
705867,Putnam,3950000.0,752.0,Putnam Hospital Center,18 to 29,NaN,M,Other Race,Unknown,4,...,Minor,Medical,Medicaid,NaN,NaN,0,N,N,13976.15,4054.04
706214,Putnam,3950000.0,752.0,Putnam Hospital Center,30 to 49,124,M,White,Not Span/Hispanic,3,...,Minor,Medical,Medicare,Medicaid,NaN,0,N,N,11769.89,3366.13
706501,Putnam,3950000.0,752.0,Putnam Hospital Center,18 to 29,105,M,Other Race,Unknown,5,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,19054.70,5566.92
706962,Putnam,3950000.0,752.0,Putnam Hospital Center,18 to 29,125,M,White,Not Span/Hispanic,10,...,Minor,Medical,Medicaid,NaN,NaN,0,N,Y,35729.45,10557.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745889,Putnam,3950000.0,752.0,Putnam Hospital Center,70 or Older,105,M,White,Not Span/Hispanic,3,...,Major,Medical,Medicare,Blue Cross/Blue Shield,NaN,0,N,Y,26577.54,7658.91
745890,Putnam,3950000.0,752.0,Putnam Hospital Center,18 to 29,105,F,White,Not Span/Hispanic,1,...,Minor,Medical,"Managed Care, Unspecified",NaN,NaN,0,N,Y,11475.15,3425.98
745891,Putnam,3950000.0,752.0,Putnam Hospital Center,30 to 49,105,M,White,Not Span/Hispanic,2,...,Moderate,Medical,Medicaid,NaN,NaN,0,N,Y,12504.90,3724.22
745892,Putnam,3950000.0,752.0,Putnam Hospital Center,70 or Older,125,F,White,Not Span/Hispanic,2,...,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,15137.95,4513.10


In [183]:
Putnam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6785 entries, 700137 to 745893
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   6785 non-null   category
 1   Operating Certificate Number      6785 non-null   float64 
 2   Facility Id                       6785 non-null   float64 
 3   Facility Name                     6785 non-null   object  
 4   Age Group                         6785 non-null   category
 5   Zip Code - 3 digits               6684 non-null   category
 6   Gender                            6785 non-null   category
 7   Race                              6785 non-null   category
 8   Ethnicity                         6785 non-null   category
 9   Length of Stay                    6785 non-null   category
 10  Type of Admission                 6785 non-null   category
 11  Patient Disposition               6785 non-null  

In [184]:
 #   simple mapping on binary features.
Putnam_df['Abortion Edit Indicator'] = Putnam_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Putnam_df['Emergency Department Indicator'] = Putnam_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [185]:
Putnam_df["Length of Stay"]=pd.to_numeric(Putnam_df["Length of Stay"],errors='coerce')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [186]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
   Putnam_df[col]= label_encoder.fit_transform(Putnam_df[col])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [187]:
Putnam_df = Putnam_df.drop("Facility Name", axis=1)

In [188]:
Putnam_df = Putnam_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)


In [189]:
Putnam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6785 entries, 700137 to 745893
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   6785 non-null   int64  
 1   Operating Certificate Number      6785 non-null   float64
 2   Facility Id                       6785 non-null   float64
 3   Age Group                         6785 non-null   int64  
 4   Zip Code - 3 digits               6785 non-null   int64  
 5   Gender                            6785 non-null   int64  
 6   Race                              6785 non-null   int64  
 7   Ethnicity                         6785 non-null   int64  
 8   Length of Stay                    6785 non-null   int64  
 9   Type of Admission                 6785 non-null   int64  
 10  Patient Disposition               6785 non-null   int64  
 11  Discharge Year                    6785 non-null   int64  
 12 

In [190]:
x = Putnam_df.drop(["Length of Stay"], axis =1)
y = Putnam_df["Length of Stay"]

In [191]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [192]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [193]:
r2 = r2_score(y_test, y_pred)
r2 


0.6833681470945818

In [194]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

1.4289408471215483

In [195]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

6.810081276526211

In [196]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.609613242709772

In [197]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [198]:
r2 = r2_score(y_test, y_pred1)
r2 

0.6496057028620932

In [199]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.5599579330447522

In [200]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

7.536240022740899

In [201]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.745221306696584

In [202]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [203]:
r2 = r2_score(y_test, y_pred3)
r2 

0.6833499677327529

In [204]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.4289867059435022

In [205]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

6.810472276138141

In [206]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.609688156875863

In [207]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [208]:
r2 = r2_score(y_test, y_pred1)
r2 


0.6496057028620932

In [209]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.5599579330447522

In [210]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

7.536240022740899

In [211]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.745221306696584

##Client 9: Nyack Hospital

In [212]:
Nyack_df=df[df["Facility Name"]== 'Nyack Hospital']

In [213]:
Nyack_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13459 entries, 700257 to 761325
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   13459 non-null  category
 1   Operating Certificate Number      13459 non-null  float64 
 2   Facility Id                       13459 non-null  float64 
 3   Facility Name                     13459 non-null  object  
 4   Age Group                         13459 non-null  category
 5   Zip Code - 3 digits               13327 non-null  category
 6   Gender                            13459 non-null  category
 7   Race                              13459 non-null  category
 8   Ethnicity                         13459 non-null  category
 9   Length of Stay                    13459 non-null  category
 10  Type of Admission                 13459 non-null  category
 11  Patient Disposition               13459 non-null

In [214]:
Nyack_df=Nyack_df.drop("Facility Name", axis=1)

In [215]:
Nyack_df = Nyack_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)

In [216]:
 #   simple mapping on binary features.
Nyack_df['Abortion Edit Indicator'] = Nyack_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Nyack_df['Emergency Department Indicator'] = Nyack_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [217]:
Nyack_df["Length of Stay"]=pd.to_numeric(Nyack_df["Length of Stay"],errors='coerce')

In [218]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
   Nyack_df[col]= label_encoder.fit_transform(Nyack_df[col])

In [219]:
Nyack_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13459 entries, 700257 to 761325
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hospital County                   13459 non-null  int64  
 1   Operating Certificate Number      13459 non-null  float64
 2   Facility Id                       13459 non-null  float64
 3   Age Group                         13459 non-null  int64  
 4   Zip Code - 3 digits               13459 non-null  int64  
 5   Gender                            13459 non-null  int64  
 6   Race                              13459 non-null  int64  
 7   Ethnicity                         13459 non-null  int64  
 8   Length of Stay                    13458 non-null  float64
 9   Type of Admission                 13459 non-null  int64  
 10  Patient Disposition               13459 non-null  int64  
 11  Discharge Year                    13459 non-null  int64  
 12

In [220]:
Nyack_df.dropna(inplace=True)

In [221]:
x = Nyack_df.drop(["Length of Stay"], axis =1)
y = Nyack_df["Length of Stay"]

In [222]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [223]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [224]:
r2 = r2_score(y_test, y_pred)
r2 


0.8349891359205954

In [225]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

1.3344919646969073

In [226]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

5.8475935175546985

In [227]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.4181797942987404

In [228]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [229]:
r2 = r2_score(y_test, y_pred1)
r2 

0.8130004942930402

In [230]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.371002670909177

In [231]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

6.626818806498411

In [232]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.574260827208154

In [233]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [234]:
r2 = r2_score(y_test, y_pred3)
r2 

0.8349875922803607

In [235]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.3343143263471502

In [236]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

5.847648220502174

In [237]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.418191105041571

##Client 10: Mount St Marys Hospital

In [238]:
#Mount St Marys Hospital and Health Center
Mount_df=df[df["Facility Name"]== 'Mount St Marys Hospital and Health Center']

In [239]:
Mount_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5314 entries, 138563 to 143964
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   5314 non-null   category
 1   Operating Certificate Number      5314 non-null   float64 
 2   Facility Id                       5314 non-null   float64 
 3   Facility Name                     5314 non-null   object  
 4   Age Group                         5314 non-null   category
 5   Zip Code - 3 digits               5267 non-null   category
 6   Gender                            5314 non-null   category
 7   Race                              5314 non-null   category
 8   Ethnicity                         5314 non-null   category
 9   Length of Stay                    5314 non-null   category
 10  Type of Admission                 5314 non-null   category
 11  Patient Disposition               5314 non-null  

In [240]:
Mount_df = Mount_df.drop("Facility Name", axis=1)


In [241]:
 #   simple mapping on binary features.
Mount_df['Abortion Edit Indicator'] = Mount_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Mount_df['Emergency Department Indicator'] = Mount_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [242]:
Mount_df["Length of Stay"]=pd.to_numeric(Mount_df["Length of Stay"],errors='coerce')

In [243]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
  Mount_df[col]= label_encoder.fit_transform(Mount_df[col])

In [244]:
Mount_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5314 entries, 138563 to 143964
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   5314 non-null   int64   
 1   Operating Certificate Number      5314 non-null   float64 
 2   Facility Id                       5314 non-null   float64 
 3   Age Group                         5314 non-null   int64   
 4   Zip Code - 3 digits               5314 non-null   int64   
 5   Gender                            5314 non-null   int64   
 6   Race                              5314 non-null   int64   
 7   Ethnicity                         5314 non-null   int64   
 8   Length of Stay                    5314 non-null   int64   
 9   Type of Admission                 5314 non-null   int64   
 10  Patient Disposition               5314 non-null   int64   
 11  Discharge Year                    5314 non-null  

In [245]:
Mount_df = Mount_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)


In [246]:
x = Mount_df.drop(["Length of Stay"], axis =1)
y = Mount_df["Length of Stay"]

In [247]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [248]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [249]:
r2 = r2_score(y_test, y_pred)
r2 


0.8493645328400692

In [250]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

1.5751957484784673

In [251]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

5.420550398990118

In [252]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.328207550668565

In [253]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [254]:
r2 = r2_score(y_test, y_pred1)
r2 


0.8032059655994213

In [255]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

7.081545945327565

In [256]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.6611174241899898

In [257]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [258]:
r2 = r2_score(y_test, y_pred3)
r2 

0.8493873434940765

In [259]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.5750406339612182

In [260]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

5.419729567734293

In [261]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.328031264337808

##Client 5: Buffalo Women & Children’s Hospital 11051


In [262]:

Buffalo_df=df[df["Facility Name"]== "Women And Children's Hospital Of Buffalo"]

In [263]:
Buffalo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13020 entries, 3309 to 54603
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Hospital County                   13020 non-null  category
 1   Operating Certificate Number      13020 non-null  float64 
 2   Facility Id                       13020 non-null  float64 
 3   Facility Name                     13020 non-null  object  
 4   Age Group                         13020 non-null  category
 5   Zip Code - 3 digits               12941 non-null  category
 6   Gender                            13020 non-null  category
 7   Race                              13020 non-null  category
 8   Ethnicity                         13020 non-null  category
 9   Length of Stay                    13020 non-null  category
 10  Type of Admission                 13020 non-null  category
 11  Patient Disposition               13020 non-null  c

In [264]:
Buffalo_df= Buffalo_df.drop("Facility Name", axis=1)

In [265]:
Buffalo_df = Buffalo_df.drop(["Payment Typology 2",    "Payment Typology 3"], axis=1)


In [266]:
 #   simple mapping on binary features.
Buffalo_df['Abortion Edit Indicator'] = Buffalo_df['Abortion Edit Indicator'].apply(
     lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))
Buffalo_df['Emergency Department Indicator'] = Buffalo_df['Emergency Department Indicator'].apply(
    lambda x: 1 if x == 'Y' else (0 if x == 'N' else None))

In [267]:
Buffalo_df["Length of Stay"]=pd.to_numeric(Buffalo_df["Length of Stay"],errors='coerce')

In [268]:
list=  [ "Hospital County","Age Group"
    ,"Gender",   
 "Race" , 
"Ethnicity", 
"Gender" , 
"Race" ,    
 "Ethnicity" ,        
 "Type of Admission"  ,  
  "Patient Disposition",      
 "Type of Admission" ,  
 "Patient Disposition"  ,
  "APR Risk of Mortality" ,   
 "APR Medical Surgical Description" ,
 "Payment Typology 1"   ,   
 "Abortion Edit Indicator"     ,  
 "Emergency Department Indicator",
       "Zip Code - 3 digits"]  

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
for col in list :
  Buffalo_df[col]= label_encoder.fit_transform(Buffalo_df[col])

In [269]:
x = Glens_df.drop(["Length of Stay"], axis =1)
y = Glens_df["Length of Stay"]

In [270]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [271]:
# Linear Regression

LR = LinearRegression()
LR.fit(x_train, y_train)
LR.score(x_train, y_train)
y_pred =LR.predict(x_test)

In [272]:
r2 = r2_score(y_test, y_pred)
r2 


0.7753107229320534

In [273]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred)
mae

1.519038644423741

In [274]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred)
mse

8.394303923962779

In [275]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.8972925161196237

In [276]:
# LASSO
L1 = linear_model.Lasso()
L1.fit(x_train,y_train)
y_pred1 = L1.predict(x_test)

In [277]:
r2 = r2_score(y_test, y_pred1)
r2 


0.7507020198871027

In [278]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred1)
mae

1.5980824331021113

In [279]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred1)
mse

9.313675490018403

In [280]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

3.0518314976450456

In [281]:
# Ridge
L2 = Ridge(alpha=1.0)
L2.fit(x_train, y_train) 
y_pred3 = L2.predict(x_test)

In [282]:
r2 = r2_score(y_test, y_pred3)
r2 

0.7753072003472253

In [283]:
# calculate the MAE of the model
mae = mean_absolute_error(y_test, y_pred3)
mae

1.5188909684172016

In [284]:
# calculate the MSE of the model
mse = mean_squared_error(y_test, y_pred3)
mse

8.394435526360677

In [285]:
# calculate the RMSE of the model
rmse = np.sqrt(mse)
rmse

2.8973152273028004